# Описание проекта
Заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России.  
Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».  
Компания провела опрос и попросила вас подготовить дашборд с его итогами. Большую базу данных для такой задачи разворачивать не стали и выгрузили данные в SQLite.  
Чтобы оценить результаты опроса, оценки обычно делят на три группы:  
9-10 баллов — «cторонники» (англ. promoters);  
7-8 баллов — «нейтралы» (англ. passives);  
0-6 баллов — «критики» (англ. detractors).  
# Данные
Таблица `user`  
Содержит основную информацию о клиентах.  
`user_id`	- идентификатор клиента, первичный ключ таблицы;  
`lt_day` - количество дней «жизни» клиента;  
`age` - возраст клиента в годах;  
`gender_segment` - пол клиента (1 – женщина, 0 – мужчина);  
`os_name` - тип операционной системы;  
`cpe_type_name` - тип устройства;  
`location_id` - идентификатор домашнего региона клиента, внешний ключ, отсылающий к таблице location;  
`age_gr_id` - идентификатор возрастного сегмента клиента, внешний ключ, отсылающий к таблице age_segment;  
`tr_gr_id`- идентификатор сегмента клиента по объёму потребляемого трафика в месяц, внешний ключ, отсылающий к таблице traffic_segment;  
`lt_gr_id` - идентификатор сегмента клиента по количеству месяцев «жизни», внешний ключ, отсылающий к таблице lifetime_segment;  
`nps_score` - оценка клиента в NPS-опросе (от 1 до 10).  

Таблица `location`  
Справочник территорий, в которых телеком-компания оказывает услуги.  
`location_id` - идентификатор записи, первичный ключ;  
`country` - страна;  
`city` - город.

Таблица `age_segment`  
Данные о возрастных сегментах клиентов.  
`age_gr_id` - идентификатор сегмента, первичный ключ;  
`bucket_min` - минимальная граница сегмента;  
`bucket_max` - максимальная граница сегмента;  
`title` - название сегмента.  

Таблица `traffic_segment`  
Данные о выделяемых сегментах по объёму потребляемого трафика.  
`tr_gr_id` - идентификатор сегмента, первичный ключ;  
`bucket_min` - минимальная граница сегмента;  
`bucket_max` - максимальная граница сегмента;  
`title` - название сегмента.  

Таблица `lifetime_segment`  
Данные о выделяемых сегментах по количеству месяцев «жизни» клиента — лайфтайму.  
`lt_gr_id` - идентификатор сегмента, первичный ключ;  
`bucket_min` - минимальная граница сегмента;  
`bucket_max` - максимальная граница сегмента;  
`title` - название сегмента.

# Ссылка на дашборд Tableau Public:
[Ссылка](https://public.tableau.com/app/profile/evgeny3184/viz/Big_project2_dashboard/Dashboard2?publish=yes)

# Подключаемся к базе

In [ ]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [ ]:
path_to_db = '/datasets/telecomm_csi.db'
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

In [ ]:
query = """
WITH

u AS (SELECT user_id,
        lt_day,
        CASE
            WHEN lt_day <= 365 THEN 'new'
            WHEN lt_day > 365 THEN 'old' END
        AS is_new,
        age,
        gender_segment,
        CASE
            WHEN CAST(gender_segment AS real) = 1 THEN 'female'
            WHEN CAST(gender_segment AS real) = 0 THEN 'male' END
        AS gender_segment,
        os_name,
        cpe_type_name,
        location_id,
        age_gr_id,
        tr_gr_id,
        lt_gr_id,
        nps_score
      FROM user),

l AS (SELECT * FROM location),

age_s AS (SELECT age_gr_id, title FROM age_segment),

tr AS (SELECT tr_gr_id, title FROM traffic_segment),

ls AS (SELECT lt_gr_id, title FROM lifetime_segment)

SELECT  u.user_id,
        u.lt_day,
        u.is_new,
        u.age,
        u.gender_segment,
        u.os_name,
        u.cpe_type_name,
        l.country,
        l.city,
        age_s.title AS age_segment,
        tr.title AS traffic_segment,
        ls.title AS lifetime_segment,
        u.nps_score,
        CASE
            WHEN u.nps_score >= 9 THEN 'protomers'
            WHEN u.nps_score = 7 OR u.nps_score = 8 THEN 'passives'
            WHEN u.nps_score <= 6 THEN 'detractors'
        END AS nps_group
FROM u
INNER JOIN l ON u.location_id = l.location_id
INNER JOIN age_s ON u.age_gr_id = age_s.age_gr_id
INNER JOIN tr ON u.tr_gr_id = tr.tr_gr_id
INNER JOIN ls ON u.lt_gr_id = ls.lt_gr_id
"""

In [ ]:
df = pd.read_sql(query, engine)
df.head(10)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,old,45.0,1.0,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,10,protomers
1,A001WF,2344,old,53.0,0.0,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,10,protomers
2,A003Q7,467,old,57.0,0.0,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,10,protomers
3,A004TB,4190,old,44.0,1.0,IOS,SMARTPHONE,Россия,РостовнаДону,04 35-44,03 0.1-1,08 36+,10,protomers
4,A004XT,1163,old,24.0,0.0,ANDROID,SMARTPHONE,Россия,Рязань,02 16-24,05 5-10,08 36+,10,protomers
5,A005O0,5501,old,42.0,1.0,ANDROID,SMARTPHONE,Россия,Омск,04 35-44,05 5-10,08 36+,6,detractors
6,A0061R,1236,old,45.0,0.0,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,06 10-15,08 36+,10,protomers
7,A009KS,313,new,35.0,0.0,ANDROID,SMARTPHONE,Россия,Москва,04 35-44,13 45-50,05 7-12,10,protomers
8,A00AES,3238,old,36.0,1.0,ANDROID,SMARTPHONE,Россия,СанктПетербург,04 35-44,04 1-5,08 36+,10,protomers
9,A00F70,4479,old,54.0,1.0,ANDROID,SMARTPHONE,Россия,Волгоград,05 45-54,07 15-20,08 36+,9,protomers


In [ ]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
d = df.assign(total_nps=lambda row: row['nps_group'].map({'protomers': 1, 'passives': 0, 'detractors': -1}))\
  .pivot_table(index=['gender_segment', 'age_segment'], columns=['lifetime_segment'], values='total_nps', aggfunc='mean')
display(d)
plt.figure(figsize=(15,8))
sns.heatmap(d, annot=True);